In [4]:
!pip install llama_index

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 609.4/609.4 kB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 5.0 MB/s eta 0:00:00
     -------------------------------------- 163.8/163.8 kB 5.0 MB/s eta 0:00:00
     ---------------------------------------- 90.0/90.0 kB 5.3 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 92.7/92.7 kB 5.2 MB/s eta 0:00:00
     ---------------------------------------- 49.4/49.4 kB ? eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [6]:
import os
import openai
import tiktoken
from dotenv import load_dotenv
import llama_index
from llama_index import GPTKeywordTableIndex,SimpleDirectoryReader,LLMPredictor,ServiceContext
from llama_index import GPTVectorStoreIndex,load_index_from_storage

load_dotenv() #read local .env file
openai.api_key = os.getenv('OPENAI_API_KEY')

In [7]:
documents = SimpleDirectoryReader('issuesubsetdata').load_data()

In [8]:
index = GPTVectorStoreIndex.from_documents(documents)

In [9]:
query_engine = index.as_query_engine()

In [12]:
response = query_engine.query("list all json schema fields")
print(response)


- "@context"
- "issue"
- "description"
- "has_author"
- "due_by"
- "status"
- "impacted_region"
- "required"


In [13]:
response = query_engine.query("list all db schema table columns")
print(response)


TestSchema.ENUMVALS
- DESCRIPTION (VARCHAR)
- ENUMVALID (DECIMAL)
- NAME (VARCHAR)

TestSchema.ISSUE
- DESCRIPTION (VARCHAR)
- IMPCTDORG (VARCHAR)
- ISSDESCR (VARCHAR)
- ISSORGNINDT (TIMESTAMP)
- ISSRORG (VARCHAR)
- NAME00 (VARCHAR)
- ORGANIZATIONREGION (VARCHAR)
- SLADUEDATE (TIMESTAMP)
- STTS (DECFLOAT)
- CREATION_DATE (TIMESTAMP)
- CREATOR (DECFLOAT)


In [24]:
query = '''
System: you are an expert of rdbms and rdf triples and json

map fields from json schema to db table columns in schema file
for the below steps to generate the output

Step 1:
add the mapped db column in the json file in the format TABLE.Column

Step 2: 
generate a SQL db view using the db columns mapped in step 1 and the db schema 
'''
response = query_engine.query(f"{query}")
print(response)


The mapping of fields from the JSON schema to the DB table columns in the schema file would be as follows:

TestSchema.ENUMVALS:
DESCRIPTION -> msrpc:Issue
ENUMVALID -> msrpc:originallyDueBy
NAME -> msrpc:IssueStatus

TestSchema.ISSUE:
DESCRIPTION -> mscore:description
IMPCTDORG -> mscore:hasAuthor
ISSDESCR -> mscore:categorizedBy
ISSORGNINDT -> msrpc:impacts
NAME00 -> mscore:description
ORGANIZATIONREGION -> msrpc:originallyDueBy
SLADUEDATE -> mscore:categorizedBy
STTS -> msrpcx:_IssueStatus_Open
CREATION_DATE -> msrpcx:_IssueStatus_Closed
CREATOR -> msrpcx:_IssueStatus_PendingVerification

The SQL view would be as follows:

CREATE VIEW TestSchema_View AS
SELECT 
    ENUMVALS.DESCRIPTION AS Issue,
    ENUMVALS.ENUMVALID AS OriginallyDueBy,
    ENUMVALS.NAME AS IssueStatus,
    ISSUE.DESCRIPTION AS Description,
    ISSUE.IMPCTDORG AS HasAuthor,
    ISSUE.ISSDESCR AS CategorizedBy,
    ISSUE.ISSORGNINDT AS Impacts,
    ISSUE.NAME00 AS Description,
    ISSUE.ORGANIZATIONREGION AS Origin